In [44]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import re
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.train import Optimizer


In [45]:
import os

def load_font_data(data_dir):

    

    image_font_pairs = []

    # Loop through each subdirectory (font folder)
    for font_dir in os.listdir(data_dir):
        if os.path.isdir(os.path.join(data_dir, font_dir)):
            # Get the font name
            font_name = font_dir

            # Get all image files in the font directory
            image_files = [f for f in os.listdir(os.path.join(data_dir, font_dir)) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

            # Select every 2nd image starting from the first one
            for i in range(0, len(image_files), 2):
                # Build the image path
                img_path = os.path.join(data_dir, font_dir, image_files[i])

                # Append (image_path, font_name) tuple to the list
                image_font_pairs.append((img_path, font_name))

    return image_font_pairs

# Example usage
data_dir = r"C:\Users\prabh_6tzckcr\Downloads\Font Dataset Large"
image_font_pairs = load_font_data(data_dir)

print(f"Loaded {len(image_font_pairs)} images with corresponding font names.")

# Show the actual font names
output_labels = set([pair[1] for pair in image_font_pairs])

print("\nFonts:")
for label, font_name in enumerate(output_labels):
    print(f"{label}: {font_name}")

Loaded 95000 images with corresponding font names.

Fonts:
0: Century
1: Futura
2: Calligraphy
3: Candara
4: Corbel
5: LCD Mono
6: Bembo
7: Myriad
8: Georgia
9: Didot
10: Bell MT
11: Hombre
12: Garamond
13: Mrs Eaves
14: Cambria
15: Algerian
16: Monotype Corsiva
17: Book Antiqua
18: Comic Sans MS
19: Brandish
20: Consolas
21: Lucida Bright
22: Courier
23: Calvin
24: Californian FB
25: Bodoni
26: Akzidenz Grotesk
27: Gill sans
28: Minion
29: Calibry
30: Franklin Gothic
31: Fascinate
32: Baskerville
33: Agency
34: Frutiger
35: Helvetica
36: Arial
37: Elephant


This function loads font data from a directory containing font images. It traverses each subdirectory within the provided data_dir, assuming each subdirectory represents a font, and collects pairs of image paths and corresponding font names.

In [46]:
import os
import random
import cv2
import numpy as np

def preprocess_images(image_font_pairs, target_size=(64, 64)):
    """
    Preprocesses a list of images by loading, converting to grayscale, resizing, and normalizing."""

    preprocessed_images = []
    font_names = []

    # Randomly select 12000 image-font pairs
    image_font_pairs_subset = random.sample(image_font_pairs, 12000)

    for img_path, font_name in image_font_pairs_subset:
        try:
            # Load the image
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Check if the image is loaded successfully
            if img is None:
                print(f"Error loading image: {img_path}")
                continue

            # Resize the image while preserving aspect ratio
            height, width = img.shape
            if height > width:
                ratio = target_size[0] / height
                new_height = target_size[0]
                new_width = int(width * ratio)
            else:
                ratio = target_size[1] / width
                new_width = target_size[1]
                new_height = int(height * ratio)
            img = cv2.resize(img, (new_width, new_height))

            # Add padding if necessary to match the target size
            pad_height = target_size[0] - new_height
            pad_width = target_size[1] - new_width
            top = pad_height // 2
            bottom = pad_height - top
            left = pad_width // 2
            right = pad_width - left
            img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)

            # Convert the image to binary format for better feature extraction
            _, img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

            # Normalize the pixel values to the range [0, 1]
            img = img / 255.0

            # Add a channel dimension to make it compatible with neural network input shape
            img = np.expand_dims(img, axis=-1)

            # Append preprocessed image and font name to the lists
            preprocessed_images.append(img)
            font_names.append(font_name)
        except Exception as e:
            print(f"Error processing image: {img_path} - {e}")

    return preprocessed_images, font_names

# Example usage
preprocessed_images, font_names = preprocess_images(image_font_pairs)

print(f"Preprocessed {len(preprocessed_images)} images with corresponding font names.")


Preprocessed 12000 images with corresponding font names.


This function preprocesses a list of images by performing the following steps:

Loading the image  

Converting to grayscale  

Resizing with aspect ratio preservation  

Adding padding if necessary to match the target size  
  
Converting to binary format for better feature extraction  

Normalizing pixel values to the range [0, 1]  

Adding a channel dimension to make it compatible with neural network input shape

In [47]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    """
    Creates a convolutional neural network (CNN) model with four convolutional layers.


    """
    model = models.Sequential()

    # Convolutional layer 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 4
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten layer
    model.add(layers.Flatten())

    # Fully connected layer
    model.add(layers.Dense(512, activation='relu'))

    # Dropout regularization
    model.add(layers.Dropout(0.5))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Define input shape and number of classes
input_shape = (64, 64, 1)  
num_classes = 40 # 40 different font classes
tf.random.set_seed(42)

# Create CNN model
cnn_model = create_cnn_model(input_shape, num_classes)

# Compile the model
cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Print model summary
cnn_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 6, 6, 128)        

This function creates a convolutional neural network (CNN) model with four convolutional layers followed by fully connected layers and an output layer for classification.

Arguments:input_shape (tuple): The shape of the input images (height, width, channels).
num_classes (int): The number of classes (i.e., font classes) for classification.  

Returns:tf.keras.Model: The CNN model. 
    
Model Summary
The created CNN model consists of four convolutional layers with max pooling, followed by a flatten layer, a fully connected layer with ReLU activation, dropout regularization, and an output layer with softmax activation.
The input shape is (64, 64, 1) indicating grayscale images of size 64x64 pixels.
The model is compiled using the Adam optimizer with sparse categorical crossentropy loss and accuracy as the metric.

In [48]:
from sklearn.model_selection import train_test_split

# Split data into training and temp sets (70% training, 30% temp)
train_images, temp_images, train_fonts, temp_fonts = train_test_split(preprocessed_images, font_names, test_size=0.3, random_state=42)

# Split temp set into validation and test sets (50% validation, 50% test)
val_images, test_images, val_fonts, test_fonts = train_test_split(temp_images, temp_fonts, test_size=0.5, random_state=0)

# Print sizes of each set
print(f"Training set: {len(train_images)} images")
print(f"Validation set: {len(val_images)} images")
print(f"Test set: {len(test_images)} images")

Training set: 8400 images
Validation set: 1800 images
Test set: 1800 images


This code snippet splits the preprocessed images and their corresponding font names into training, validation, and test sets using the train_test_split function from scikit-learn.

The initial split is performed with a test size of 30%, resulting in 70% of the data used for training and 30% for further splitting.

The temp set obtained from the initial split is further split into validation and test sets with a test size of 50% each, resulting in equal proportions for validation and test sets.

The random_state parameter ensures reproducibility by fixing the random seed for the splitting process.

Output

After splitting, the code prints the sizes of each set, including the number of images in the training, validation, and test sets.

In [49]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels for training data
train_labels_encoded = label_encoder.fit_transform(train_fonts)

# Transform labels for validation and test data
val_labels_encoded = label_encoder.transform(val_fonts)
test_labels_encoded = label_encoder.transform(test_fonts)


The fit_transform method is used on the training font names (train_fonts) to both fit the encoder to the unique font names and transform them into numerical labels (train_labels_encoded). 

For the validation and test sets, the transform method is used to transform the font names into corresponding numerical labels without refitting the encoder. 

Output
The encoded labels for the training, validation, and test sets are stored in the variables train_labels_encoded, val_labels_encoded, and test_labels_encoded, respectively.

In [50]:
# Convert train_images, val_images, test_images to NumPy arrays
train_images_np = np.array(train_images)
val_images_np = np.array(val_images)
test_images_np = np.array(test_images)

# Convert train_fonts, val_fonts, test_fonts to NumPy arrays
train_fonts_np = np.array(train_labels_encoded)
val_fonts_np = np.array(val_labels_encoded)
test_fonts_np = np.array(test_labels_encoded)

In [51]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Assuming your model is already defined

# Prepare callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=2)  # Stop training if val_loss doesn't improve for 2 epochs
model_checkpoint = ModelCheckpoint("C:/Users/prabh_6tzckcr/Downloads/Keras/best_model.keras", save_best_only=True)  # Save the best model based on val_loss


# Train the model with callbacks
history = cnn_model.fit(train_images_np, train_fonts_np, epochs=20,
                        validation_data=(val_images_np, val_fonts_np),
                        callbacks=[early_stopping, model_checkpoint])  # Pass callbacks as a list

# Evaluate the model on the test set
test_loss, test_accuracy = cnn_model.evaluate(test_images_np, test_fonts_np)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/20
263/263 [==============================] - 56s 197ms/step - loss: 3.5067 - accuracy: 0.0500 - val_loss: 2.9988 - val_accuracy: 0.1283
Epoch 2/20
263/263 [==============================] - 49s 186ms/step - loss: 2.8503 - accuracy: 0.1437 - val_loss: 2.5252 - val_accuracy: 0.2122
Epoch 3/20
263/263 [==============================] - 50s 188ms/step - loss: 2.4403 - accuracy: 0.2454 - val_loss: 2.0929 - val_accuracy: 0.3239
Epoch 4/20
263/263 [==============================] - 52s 199ms/step - loss: 2.0589 - accuracy: 0.3461 - val_loss: 1.7376 - val_accuracy: 0.4444
Epoch 5/20
263/263 [==============================] - 49s 185ms/step - loss: 1.7941 - accuracy: 0.4187 - val_loss: 1.7008 - val_accuracy: 0.4539
Epoch 6/20
263/263 [==============================] - 56s 214ms/step - loss: 1.6092 - accuracy: 0.4663 - val_loss: 1.5306 - val_accuracy: 0.4994
Epoch 7/20
263/263 [==============================] - 81s 305ms/step - loss: 1.4729 - accuracy: 0.5043 - val_loss: 1.4584 - val_ac

In [52]:
from sklearn.metrics import classification_report

# Calculate predicted labels for the test set
predicted_labels = np.argmax(cnn_model.predict(test_images_np), axis=1)

# Generate classification report
report = classification_report(test_fonts_np, predicted_labels)
print("Classification Report:\n", report)

57/57 [==============================] - 4s 56ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.90      1.00      0.95        44
           1       0.24      0.14      0.17        44
           2       0.80      0.93      0.86        59
           3       0.61      0.57      0.59        44
           4       0.37      0.31      0.34        42
           5       0.33      0.16      0.21        45
           6       0.67      0.63      0.65        54
           7       0.70      0.58      0.63        52
           8       0.36      0.20      0.26        44
           9       0.91      0.93      0.92        42
          10       0.38      0.27      0.31        45
          11       0.36      0.54      0.43        46
          12       0.86      0.78      0.82        55
          13       0.83      0.95      0.89        42
          14       0.57      0.25      0.35        32
          15       0.31      0.35      0.33        49
   

In [53]:
from keras.models import load_model
model = load_model("C:/Users/prabh_6tzckcr/Downloads/Keras/best_model.keras")